<a href="https://colab.research.google.com/github/derek881107/Real-Time-Disaster-Detection-System/blob/main/real_time_disaster_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Enhanced GDACS Supplier Impact Analysis System — Overview
==============================================================

What this program does
----------------------
This program monitors global disasters through GDACS (and USGS as backup)
and analyzes which suppliers could be affected. It:

1. Loads a supplier list from a CSV file.
2. Uses the Google Maps API to geocode supplier addresses.
3. Fetches disaster alerts from GDACS within a user-defined time window.
4. Falls back to USGS earthquake data when needed.
5. Calculates distances between disasters and supplier locations.
6. Classifies supplier impact levels (Critical / High / Medium / Low).
7. Sends a summary email via Gmail and saves a full JSON report.

Why it’s important
------------------
• Provides supply chain teams with a quick email alert about which suppliers
  may be affected.  
• Creates a detailed JSON report for dashboards, audits, or deeper analysis.  
• Works end-to-end with just a supplier CSV, a Google Maps API key, and a
  Gmail App Password.

Key features
------------
• Parses GDACS alerts with complete metadata (type, severity, coordinates, etc.).  
• Multiple coordinate extraction methods with regex fallbacks.  
• USGS backup feed for earthquake events.  
• Haversine distance-based impact scoring.  
• Email summaries with grouped supplier impacts.  
• JSON reports with metadata, statistics, alerts, and supplier details.

What you need before running
----------------------------
• Python 3.8+  
• `requests` package  
• Google Maps API key (billing enabled)  
• Gmail account + Gmail App Password  
• Supplier file named `merged_full_dataset.csv`

CSV format required
-------------------
The supplier CSV must include:
- id
- name
- address
- country
- contact_person
- phone
- email
- product_category
- critical_level
- type

The script auto-detects delimiters (comma, pipe, semicolon, tab).

Workflow
--------
1. Prompt for Gmail and Google Maps API credentials.  
2. Load and geocode supplier addresses.  
3. Select monitoring period (days) and minimum alert level (Green, Orange, Red).  
4. Retrieve alerts from GDACS and USGS.  
5. Compute disaster–supplier distances.  
6. Assign impact levels based on radius thresholds.  
7. Output results:
   • Email: readable disaster + supplier summary.  
   • JSON: detailed report with metadata and impact analysis.

Defaults
--------
• Monitoring period: 7 days (max 30).  
• Minimum alert level: Green.  
• Impact radius: 100 km.  
• Supplier CSV: `merged_full_dataset.csv`  
• SMTP: Gmail TLS on port 587.

Limitations
-----------
• Gmail requires an App Password (not the regular login password).  
• Google Maps API requires billing and quota.  
• Address errors or quota issues prevent geocoding.  
• Secondary effects (e.g., logistics disruptions) are not modeled.  
• USGS backup only supports earthquakes.

Security notes
--------------
• Do not hardcode credentials in the script or repo.  
• Use interactive input (as provided) or environment variables.  
• Rotate API keys and passwords regularly.

Ideas for extension
-------------------
• Add more disaster data sources.  
• Save geocoding results to a database.  
• Integrate Slack/Teams notifications.  
• Attach CSV/JSON reports to emails.  
• Automate with schedulers (e.g., daily run).

Quick start
-----------
1. Place `merged_full_dataset.csv` in the working directory.  
2. Run the script.  
3. Enter Gmail, Gmail App Password, recipient email, and Maps API key.  
4. Select monitoring days, minimum alert level, and impact radius.  
5. Check your inbox for the alert email and the directory for the JSON report.


In [5]:
#!/usr/bin/env python3
"""
Enhanced GDACS System with Complete Data Extraction and Supplier Impact Analysis
Monitors fixed supplier list, analyzes disaster impact on suppliers, obtains complete GDACS data
"""

import smtplib
from email.mime.text import MIMEText
from email.header import Header
from email.utils import formataddr
from datetime import datetime, timedelta
import requests
import json
import sys
import csv
import os
import time
import xml.etree.ElementTree as ET
import re
from urllib.parse import quote
import math


class EnhancedGDACSSupplierImpactSystem:
    def __init__(self):
        self.email_config = None
        self.google_maps_api_key = None
        self.suppliers = []
        print("✅ Enhanced GDACS Supplier Impact Analysis System Initialized")
        print("🏭 Features: Complete GDACS data extraction + Supplier impact analysis")
        print("📊 Enhanced with full GDACS attributes support")

    def setup_gmail(self, email, password, recipient):
        """Setup Gmail configuration"""
        self.email_config = {
            'server': 'smtp.gmail.com',
            'port': 587,
            'email': email.strip(),
            'password': password.strip(),
            'recipient': recipient.strip()
        }
        print(f"📨 Sender: {self.email_config['email']}")
        print(f"📥 Recipient: {self.email_config['recipient']}")
        return True

    def setup_google_maps(self, api_key):
        """Setup Google Maps API key"""
        self.google_maps_api_key = api_key.strip()
        print("🗺️ Google Maps API configured")
        return True

    def validate_csv_format(self, csv_file_path):
        """Validate CSV format and provide correction suggestions"""
        if not os.path.exists(csv_file_path):
            print(f"❌ CSV file not found: {csv_file_path}")
            return False

        try:
            with open(csv_file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                lines = content.split('\n')

                print("\n🔍 CSV Format Analysis:")
                print("="*50)

                # Check first few lines
                for i, line in enumerate(lines[:3], 1):
                    if line.strip():
                        comma_count = line.count(',')
                        pipe_count = line.count('|')
                        semicolon_count = line.count(';')
                        quote_count = line.count('"')

                        print(f"Line {i}: {line[:80]}{'...' if len(line) > 80 else ''}")
                        print(f"   Commas: {comma_count}, Pipes: {pipe_count}, Semicolons: {semicolon_count}, Quotes: {quote_count}")

                # Reset file pointer and test parsing
                file.seek(0)

                try:
                    reader = csv.DictReader(file)
                    header = reader.fieldnames
                    first_row = next(reader, None)

                    if header and first_row:
                        print(f"\n✅ Standard CSV parsing successful")
                        print(f"   Headers: {header}")
                        print(f"   Field count: {len(header)}")

                        # Check if address field is properly parsed
                        if 'address' in first_row:
                            address = first_row['address']
                            if len(address) < 10:
                                print(f"⚠️ Address seems too short: '{address}'")
                                print(f"   This might indicate CSV parsing issues")
                            else:
                                print(f"✅ Address field looks good: '{address[:50]}...'")

                        return True

                except Exception as e:
                    print(f"❌ Standard CSV parsing failed: {e}")
                    return False

        except Exception as e:
            print(f"❌ Error analyzing CSV: {e}")
            return False

    def load_suppliers_from_csv(self, csv_file_path):
        """Load supplier list from CSV file - supports addresses with commas"""
        if not os.path.exists(csv_file_path):
            print(f"❌ CSV file not found: {csv_file_path}")
            return False

        try:
            suppliers = []

            with open(csv_file_path, 'r', encoding='utf-8') as file:
                # Detect delimiter
                sample = file.read(1024)
                file.seek(0)

                if sample.count('|') > sample.count(','):
                    delimiter = '|'
                    print("📝 Detected delimiter: | (pipe)")
                elif sample.count(';') > sample.count(','):
                    delimiter = ';'
                    print("📝 Detected delimiter: ; (semicolon)")
                elif sample.count('\t') > 0:
                    delimiter = '\t'
                    print("📝 Detected delimiter: \\t (tab)")
                else:
                    delimiter = ','
                    print("📝 Detected delimiter: , (comma) - Using quote protection")

                # Read CSV with appropriate settings
                if delimiter == ',':
                    reader = csv.DictReader(file, delimiter=delimiter, quotechar='"', quoting=csv.QUOTE_MINIMAL)
                else:
                    reader = csv.DictReader(file, delimiter=delimiter)

                for row_num, row in enumerate(reader, 1):
                    # Clean field values
                    cleaned_row = {}
                    for key, value in row.items():
                        if key:
                            cleaned_key = key.strip()
                            cleaned_value = str(value).strip() if value else ''
                            cleaned_row[cleaned_key] = cleaned_value

                    supplier = {
                        'id': cleaned_row.get('id', f'SUP_{row_num:03d}'),
                        'name': cleaned_row.get('name', ''),
                        'address': cleaned_row.get('address', ''),
                        'country': cleaned_row.get('country', ''),
                        'contact_person': cleaned_row.get('contact_person', ''),
                        'phone': cleaned_row.get('phone', ''),
                        'email': cleaned_row.get('email', ''),
                        'product_category': cleaned_row.get('product_category', ''),
                        'critical_level': cleaned_row.get('critical_level', 'Medium'),
                        'type': cleaned_row.get('type', ''),
                        'latitude': None,
                        'longitude': None,
                        'geocoded': False
                    }

                    # Validate required fields
                    if supplier['name'] and supplier['address']:
                        suppliers.append(supplier)
                        short_address = supplier['address'][:50] + "..." if len(supplier['address']) > 50 else supplier['address']
                        print(f"✅ Loaded: {supplier['name']} ({short_address}) - Type: {supplier['type']}")
                    else:
                        print(f"⚠️ Row {row_num}: Missing name or address, skipped")

            self.suppliers = suppliers
            print(f"📊 Total suppliers loaded: {len(self.suppliers)}")
            return len(self.suppliers) > 0

        except Exception as e:
            print(f"❌ Error loading CSV: {e}")
            return False

    def geocode_suppliers(self):
        """Convert supplier addresses to latitude/longitude coordinates"""
        if not self.google_maps_api_key:
            print("❌ Google Maps API key not configured")
            return False

        if not self.suppliers:
            print("❌ No suppliers loaded")
            return False

        print("🗺️ Geocoding supplier addresses...")
        geocoded_count = 0

        for i, supplier in enumerate(self.suppliers, 1):
            if supplier['geocoded']:
                print(f"   {i:2d}. {supplier['name']}: Already geocoded")
                geocoded_count += 1
                continue

            try:
                address = f"{supplier['address']}, {supplier['country']}"
                url = f"https://maps.googleapis.com/maps/api/geocode/json?address={quote(address)}&key={self.google_maps_api_key}"

                response = requests.get(url, timeout=10)
                data = response.json()

                if data.get('status') == 'OK' and data.get('results'):
                    location = data['results'][0]['geometry']['location']
                    supplier['latitude'] = location['lat']
                    supplier['longitude'] = location['lng']
                    supplier['geocoded'] = True
                    geocoded_count += 1

                    print(f"   {i:2d}. {supplier['name']}: ✅ ({location['lat']:.4f}, {location['lng']:.4f})")
                else:
                    print(f"   {i:2d}. {supplier['name']}: ❌ Geocoding failed ({data.get('status')})")

                # API rate limiting
                time.sleep(0.2)

            except Exception as e:
                print(f"   {i:2d}. {supplier['name']}: ❌ Error - {e}")

        print(f"📊 Geocoding complete: {geocoded_count}/{len(self.suppliers)} suppliers")
        return geocoded_count > 0

    def get_enhanced_gdacs_alerts(self, days=7, min_alert_level='Green'):
        """Get complete GDACS disaster alert data with level filtering"""
        print(f"🔍 Fetching enhanced GDACS alerts from past {days} days...")
        print(f"🚨 Minimum alert level: {min_alert_level}")
        alerts = []

        try:
            # Try GDACS RSS feed first
            rss_url = "https://www.gdacs.org/xml/rss.xml"
            print(f"📡 Connecting to: {rss_url}")

            response = requests.get(rss_url, timeout=30)
            response.raise_for_status()

            root = ET.fromstring(response.content)
            since = datetime.now() - timedelta(days=days)
            items = root.findall('.//item')
            print(f"📊 Found {len(items)} total RSS items")

            for item in items:
                try:
                    alert_data = self._parse_enhanced_rss_item(item, since)
                    if alert_data and self._meets_alert_level_criteria(alert_data['alert_level'], min_alert_level):
                        alerts.append(alert_data)
                        print(f"✅ Added: {alert_data['event_type']} ({alert_data['event_type_short']}) in {alert_data['country']} - {alert_data['alert_level']} Alert")
                    elif alert_data:
                        print(f"⚪ Filtered: {alert_data['event_type']} ({alert_data['event_type_short']}) in {alert_data['country']} - {alert_data['alert_level']} Alert (below {min_alert_level} threshold)")
                except Exception as e:
                    print(f"⚠️ Error parsing RSS item: {e}")
                    continue

            if not alerts:
                print("🔄 RSS returned no qualifying alerts, trying backup methods...")
                alerts = self._get_enhanced_backup_alerts(days, min_alert_level)

        except Exception as e:
            print(f"❌ Error fetching RSS: {e}")
            print("🔄 Trying backup methods...")
            alerts = self._get_enhanced_backup_alerts(days, min_alert_level)

        print(f"📈 Total enhanced alerts found (>= {min_alert_level}): {len(alerts)}")
        return alerts

    def _parse_enhanced_rss_item(self, item, since_date):
        """Parse RSS item and extract all GDACS attributes according to official API spec"""
        try:
            namespaces = {
                'gdacs': 'http://www.gdacs.org',
                'georss': 'http://www.georss.org/georss',
                'geo': 'http://www.w3.org/2003/01/geo/wgs84_pos#'
            }

            # Basic attributes
            title = self._get_text_element(item, 'title', 'Unknown Event')
            description = self._get_text_element(item, 'description', '')
            pub_date = self._get_text_element(item, 'pubDate', '')
            link = self._get_text_element(item, 'link', '')
            guid = self._get_text_element(item, 'guid', '')

            # Parse publication date
            event_date = self._parse_pub_date(pub_date)
            if event_date and event_date < since_date:
                return None

            # GDACS specific attributes (all fields from official API)
            gdacs_data = {}
            gdacs_elements = [
                'alertlevel',      # Alert level ("Red", "Orange", "Green")
                'country',         # Country where incident happened
                'durationinweek',  # Duration of the incident in full weeks
                'eventid',         # Event ID (numerical)
                'eventname',       # Short event name
                'eventtype',       # Short event type ("DR", "EQ", "FL", "TC", "TS", "VO", "WF")
                'fromdate',        # Date and time this incident started
                'icon',            # Icon URL
                'iscurrent',       # Whether this incident is current
                'population',      # Exposed population
                'severity',        # Severity of the incident
                'temporary',       # Whether this incident is temporary
                'todate',          # Date and time this incident ended
                'version',         # Version of the incident in this feed
                'vulnerability'    # Vulnerability score (textual or numerical)
            ]

            for element in gdacs_elements:
                value = self._get_text_element(item, f'gdacs:{element}', None, namespaces)
                if value:
                    gdacs_data[element] = value

            # Geographic information (geometries and coordinates)
            coordinates = self._extract_enhanced_coordinates(item, namespaces)

            # Process event type (convert short to long form)
            event_type_short = gdacs_data.get('eventtype', 'UN')
            event_type = self._get_full_event_type(event_type_short)

            # Process alert level (GDACS standard: Green, Orange, Red)
            alert_level = gdacs_data.get('alertlevel', 'Unknown')
            if not alert_level or alert_level == 'Unknown':
                alert_level = self._extract_alert_level_from_text(title, description)

            # Process country
            country = gdacs_data.get('country', 'Unknown')
            if not country or country == 'Unknown':
                country = self._extract_country_from_text(title, description)

            # Get location name (distance_to_home equivalent)
            location_name = "Unknown Location"
            if coordinates['latitude'] and coordinates['longitude']:
                location_name = self._get_location_name(coordinates['latitude'], coordinates['longitude'])

            # Build complete alert data with all GDACS API fields
            enhanced_alert = {
                # Basic fields
                'title': title,
                'description': description,
                'external_id': guid,
                'attribution': 'GDACS',

                # Geographic data
                'geometries': coordinates,  # All geometry details
                'coordinates': coordinates,  # Best coordinates
                'location_name': location_name,

                # Alert classification
                'category': alert_level,
                'alert_level': alert_level,  # Alert level ("Red", "Orange", "Green")

                # Event information
                'event_type': event_type,
                'event_type_short': event_type_short,  # Short event type
                'event_id': self._safe_int(gdacs_data.get('eventid')),  # Event ID (numerical)
                'event_name': gdacs_data.get('eventname', title),  # Short event name

                # Location
                'country': country,  # Country where incident happened

                # Severity and impact
                'severity': gdacs_data.get('severity', 'Unknown'),  # Severity of the incident
                'population': self._safe_int(gdacs_data.get('population')),  # Exposed population
                'vulnerability': gdacs_data.get('vulnerability', 'Unknown'),  # Vulnerability score

                # Timing
                'from_date': self._parse_gdacs_date(gdacs_data.get('fromdate')),  # Date/time started
                'to_date': self._parse_gdacs_date(gdacs_data.get('todate')),  # Date/time ended
                'pub_date': event_date.strftime('%Y-%m-%d %H:%M:%S') if event_date else '',
                'duration_in_week': self._safe_int(gdacs_data.get('durationinweek')),  # Duration in weeks

                # Status flags
                'is_current': gdacs_data.get('iscurrent', '').lower() == 'true',  # Whether current
                'temporary': gdacs_data.get('temporary', '').lower() == 'true',  # Whether temporary

                # Metadata
                'icon_url': gdacs_data.get('icon', ''),  # Icon URL
                'version': gdacs_data.get('version', '1'),  # Version in feed
                'link': link,
                'source': 'GDACS_ENHANCED'
            }

            return enhanced_alert

        except Exception as e:
            print(f"⚠️ Error parsing enhanced RSS item: {e}")
            return None

    def _get_enhanced_backup_alerts(self, days, min_alert_level='Green'):
        """Enhanced backup alert retrieval using USGS with level filtering"""
        print("🌍 Using enhanced backup data sources...")
        alerts = []
        alerts.extend(self._get_usgs_earthquake_data(days, min_alert_level))
        return alerts

    def _get_usgs_earthquake_data(self, days, min_alert_level='Green'):
        """Get USGS earthquake data and convert to GDACS format with level filtering"""
        alerts = []

        try:
            usgs_url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_week.geojson"
            response = requests.get(usgs_url, timeout=30)
            data = response.json()

            since = datetime.now() - timedelta(days=days)

            for feature in data.get('features', []):
                try:
                    props = feature.get('properties', {})
                    coords = feature.get('geometry', {}).get('coordinates', [])

                    timestamp = props.get('time')
                    if timestamp:
                        event_date = datetime.fromtimestamp(timestamp / 1000)
                        if event_date < since:
                            continue
                    else:
                        continue

                    magnitude = props.get('mag', 0)
                    if magnitude < 4.5:  # Only focus on larger earthquakes
                        continue

                    # Determine alert level (GDACS standard: Green, Orange, Red)
                    if magnitude >= 7.0:
                        alert_level = 'Red'
                    elif magnitude >= 6.0:
                        alert_level = 'Orange'
                    else:
                        alert_level = 'Green'

                    # Check if alert meets minimum level criteria
                    if not self._meets_alert_level_criteria(alert_level, min_alert_level):
                        continue

                    location_name = "Unknown Location"
                    country = "Unknown"
                    if len(coords) > 1:
                        location_name = self._get_location_name(coords[1], coords[0])
                        country = self._extract_country_from_place(props.get('place', ''))

                    earthquake_alert = {
                        'title': props.get('title', 'Earthquake Event'),
                        'description': f"Magnitude {magnitude} earthquake - {props.get('place', 'Unknown location')}",
                        'external_id': feature.get('id', ''),
                        'attribution': 'USGS',
                        'coordinates': {
                            'latitude': coords[1] if len(coords) > 1 else None,
                            'longitude': coords[0] if len(coords) > 0 else None
                        },
                        'location_name': location_name,
                        'category': alert_level,
                        'alert_level': alert_level,
                        'event_type': 'Earthquake',
                        'event_type_short': 'EQ',
                        'country': country,
                        'event_id': None,
                        'event_name': f"M{magnitude} {props.get('place', 'Earthquake')}",
                        'severity': str(magnitude),
                        'population': None,
                        'vulnerability': 'Unknown',
                        'from_date': event_date,
                        'to_date': None,
                        'pub_date': event_date.strftime('%Y-%m-%d %H:%M:%S'),
                        'duration_in_week': None,
                        'is_current': True,
                        'temporary': False,
                        'icon_url': '',
                        'version': '1',
                        'link': props.get('url', ''),
                        'source': 'USGS_ENHANCED',
                        'magnitude': magnitude,
                        'depth': coords[2] if len(coords) > 2 else None
                    }

                    alerts.append(earthquake_alert)

                except Exception as e:
                    continue

            print(f"🌍 USGS enhanced found {len(alerts)} earthquake alerts (>= {min_alert_level})")

        except Exception as e:
            print(f"❌ Enhanced backup failed: {e}")

        return alerts

    def _meets_alert_level_criteria(self, alert_level, min_alert_level):
        """Check if alert level meets minimum criteria"""
        # Define GDACS alert level hierarchy (higher number = more severe)
        # Official GDACS levels: Green, Orange, Red only
        level_hierarchy = {
            'Green': 1,
            'Orange': 2,
            'Red': 3,
            'Unknown': 0  # Unknown alerts are included at Green level
        }

        alert_value = level_hierarchy.get(alert_level, 0)
        min_value = level_hierarchy.get(min_alert_level, 1)

        # For Unknown alerts, include them if minimum is Green
        if alert_level == 'Unknown' and min_alert_level == 'Green':
            return True

        return alert_value >= min_value

    def _get_alert_level_description(self, level):
        """Get description for alert level"""
        descriptions = {
            'Green': 'Green and above (All alerts)',
            'Orange': 'Orange and above (High to severe)',
            'Red': 'Red only (Severe only)'
        }
        return descriptions.get(level, level)

    def analyze_supplier_impact(self, disaster_lat, disaster_lon, impact_radius_km=100):
        """Analyze disaster impact on suppliers"""
        if not self.suppliers:
            return []

        impacted_suppliers = []

        for supplier in self.suppliers:
            if not supplier['geocoded']:
                continue

            distance = self._calculate_distance(
                disaster_lat, disaster_lon,
                supplier['latitude'], supplier['longitude']
            )

            if distance <= impact_radius_km:
                impact_level = self._determine_impact_level(distance, impact_radius_km)

                supplier_impact = supplier.copy()
                supplier_impact['distance_km'] = distance
                supplier_impact['impact_level'] = impact_level

                impacted_suppliers.append(supplier_impact)

        # Sort by distance
        impacted_suppliers.sort(key=lambda x: x['distance_km'])
        return impacted_suppliers

    def send_enhanced_supplier_impact_alert(self, alerts, impact_radius=100, days=7, min_alert_level='Green'):
        """Send enhanced supplier impact analysis report and save JSON"""
        if not self.email_config:
            print("❌ Email configuration not set")
            return False

        if not alerts:
            print("ℹ️ No alerts to send")
            return self._send_no_alerts_notification(days, min_alert_level)

        try:
            # Generate JSON report
            json_data = self._generate_json_report(alerts, impact_radius, days, min_alert_level)
            json_saved = self._save_json_report(json_data)

            # Build enhanced email content
            body = self._build_email_body(alerts, impact_radius, days, min_alert_level)

            # Build and send email
            total_impacted = self._count_total_impacted_suppliers(alerts, impact_radius)
            subject = f"🚨 Enhanced Supplier Impact Alert: {total_impacted} Suppliers Affected | {len(alerts)} Disasters ({days} days, {min_alert_level}+ level)"

            msg = MIMEText(body, 'plain', 'utf-8')
            msg['Subject'] = Header(subject, 'utf-8')
            msg['From'] = formataddr(("Enhanced GDACS Supplier Impact System", self.email_config['email']))
            msg['To'] = formataddr(("Supply Chain Manager", self.email_config['recipient']))

            server = smtplib.SMTP(self.email_config['server'], self.email_config['port'])
            server.starttls()
            server.login(self.email_config['email'], self.email_config['password'])
            server.sendmail(self.email_config['email'], self.email_config['recipient'], msg.as_string())
            server.quit()

            print("📧 Enhanced supplier impact alert email sent successfully!")
            if json_saved:
                print("💾 JSON report saved successfully!")
            return True

        except Exception as e:
            print(f"❌ Failed to send enhanced supplier impact alert: {e}")
            return False

    def send_test_email(self):
        """Send test email"""
        if not self.email_config:
            print("❌ Email configuration not set")
            return False

        try:
            body = "✅ Enhanced GDACS Supplier Impact System test is working!\n"
            body += f"📅 Test time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
            body += f"🏭 Monitoring {len(self.suppliers)} suppliers\n"
            body += f"🗺️ Google Maps API: {'✅ Configured' if self.google_maps_api_key else '❌ Not configured'}\n"
            body += f"🔧 Enhanced Features:\n"
            body += f"   • Complete GDACS attribute extraction\n"
            body += f"   • Multiple backup data sources\n"
            body += f"   • Enhanced coordinate parsing\n"
            body += f"   • Improved event classification\n"
            body += f"   • Supplier type analysis support\n"
            body += f"   • User-defined monitoring period\n"

            msg = MIMEText(body, 'plain', 'utf-8')
            msg['Subject'] = Header("✅ Enhanced GDACS Supplier System Test", 'utf-8')
            msg['From'] = formataddr(("Enhanced GDACS Supplier Impact System", self.email_config['email']))
            msg['To'] = formataddr(("Supply Chain Manager", self.email_config['recipient']))

            server = smtplib.SMTP(self.email_config['server'], self.email_config['port'])
            server.starttls()
            server.login(self.email_config['email'], self.email_config['password'])
            server.sendmail(self.email_config['email'], self.email_config['recipient'], msg.as_string())
            server.quit()

            print("📬 Enhanced test email sent successfully!")
            return True

        except Exception as e:
            print(f"❌ Error sending test email: {e}")
            return False

    # Helper methods
    def _get_text_element(self, item, tag, default='', namespaces=None):
        """Safely get XML element text"""
        try:
            if namespaces:
                element = item.find(tag, namespaces)
            else:
                element = item.find(tag)
            return element.text if element is not None and element.text else default
        except:
            return default

    def _parse_pub_date(self, pub_date):
        """Parse publication date"""
        if not pub_date:
            return datetime.now()

        try:
            formats = [
                "%a, %d %b %Y %H:%M:%S %Z",
                "%a, %d %b %Y %H:%M:%S",
                "%Y-%m-%d %H:%M:%S",
                "%Y-%m-%dT%H:%M:%SZ",
                "%Y-%m-%dT%H:%M:%S"
            ]

            for fmt in formats:
                try:
                    return datetime.strptime(pub_date[:len(fmt.replace('%Z', ''))], fmt.replace('%Z', ''))
                except:
                    continue
        except:
            pass

        return datetime.now()

    def _parse_gdacs_date(self, date_str):
        """Parse GDACS date format"""
        if not date_str:
            return None

        try:
            formats = [
                "%Y-%m-%dT%H:%M:%SZ",
                "%Y-%m-%d %H:%M:%S",
                "%Y-%m-%d"
            ]

            for fmt in formats:
                try:
                    return datetime.strptime(date_str[:len(fmt.replace('Z', ''))], fmt.replace('Z', ''))
                except:
                    continue
        except:
            pass

        return None

    def _safe_int(self, value):
        """Safely convert to integer"""
        try:
            if value:
                return int(float(value))
        except:
            pass
        return None

    def _get_full_event_type(self, short_type):
        """Convert short event type to full name"""
        type_mapping = {
            'DR': 'Drought',
            'EQ': 'Earthquake',
            'FL': 'Flood',
            'TC': 'Tropical Cyclone',
            'TS': 'Tsunami',
            'VO': 'Volcano',
            'WF': 'Wild Fire',
            'ST': 'Storm',
            'CY': 'Cyclone'
        }
        return type_mapping.get(short_type, short_type)

    def _extract_enhanced_coordinates(self, item, namespaces):
        """Extract enhanced coordinate information"""
        coordinates = {'latitude': None, 'longitude': None}

        try:
            # Try to get from georss:point
            point = self._get_text_element(item, 'georss:point', '', namespaces)
            if point:
                parts = point.strip().split()
                if len(parts) >= 2:
                    coordinates['latitude'] = float(parts[0])
                    coordinates['longitude'] = float(parts[1])
                    return coordinates

            # Try to get from geo:lat and geo:long
            lat = self._get_text_element(item, 'geo:lat', '', namespaces)
            lon = self._get_text_element(item, 'geo:long', '', namespaces)
            if lat and lon:
                coordinates['latitude'] = float(lat)
                coordinates['longitude'] = float(lon)
                return coordinates

            # Extract from description text
            description = self._get_text_element(item, 'description', '')
            extracted = self._extract_coordinates_from_text(description)
            if extracted['latitude'] and extracted['longitude']:
                return extracted

        except Exception as e:
            print(f"⚠️ Error extracting coordinates: {e}")

        return coordinates

    def _extract_coordinates_from_text(self, text):
        """Extract coordinates from text"""
        coordinates = {'latitude': None, 'longitude': None}

        try:
            patterns = [
                r'[-+]?\d+\.\d+[,\s]+[-+]?\d+\.\d+',
                r'\([-+]?\d+\.\d+[,\s]+[-+]?\d+\.\d+\)',
                r'lat[:\s]+[-+]?\d+\.\d+[,\s]+lon[:\s]+[-+]?\d+\.\d+',
                r'latitude[:\s]+[-+]?\d+\.\d+[,\s]+longitude[:\s]+[-+]?\d+\.\d+'
            ]

            for pattern in patterns:
                match = re.search(pattern, text, re.IGNORECASE)
                if match:
                    coord_str = match.group()
                    parsed = self._parse_coordinate_string(coord_str)
                    if parsed['latitude'] and parsed['longitude']:
                        return parsed
        except Exception as e:
            print(f"⚠️ Coordinate extraction error: {e}")

        return coordinates

    def _parse_coordinate_string(self, coord_str):
        """Parse coordinate string"""
        try:
            coord_str = coord_str.replace('(', '').replace(')', '').replace('lat:', '').replace('lon:', '')
            coord_str = coord_str.replace('latitude:', '').replace('longitude:', '')

            numbers = re.findall(r'[-+]?\d+\.\d+', coord_str)

            if len(numbers) >= 2:
                lat = float(numbers[0])
                lon = float(numbers[1])

                if -90 <= lat <= 90 and -180 <= lon <= 180:
                    return {'latitude': lat, 'longitude': lon}
        except:
            pass

        return {'latitude': None, 'longitude': None}

    def _extract_alert_level_from_text(self, title, description):
        """Extract alert level from text (GDACS standard: Green, Orange, Red)"""
        text = f"{title} {description}".lower()

        if any(word in text for word in ['red', 'severe', 'extreme', 'critical']):
            return 'Red'
        elif any(word in text for word in ['orange', 'high', 'major']):
            return 'Orange'
        elif any(word in text for word in ['green', 'low', 'minor', 'medium', 'moderate']):
            return 'Green'
        else:
            return 'Unknown'

    def _extract_country_from_text(self, title, description):
        """Extract country from text"""
        text = f"{title} {description}"

        countries = [
            'China', 'Japan', 'Indonesia', 'Philippines', 'India', 'Turkey', 'Iran', 'Pakistan',
            'Chile', 'Peru', 'Mexico', 'Italy', 'Greece', 'Afghanistan', 'Nepal', 'Myanmar',
            'Thailand', 'Vietnam', 'Taiwan', 'South Korea', 'North Korea', 'Malaysia', 'Singapore',
            'Australia', 'New Zealand', 'Russia', 'United States', 'Canada', 'Brazil', 'Argentina',
            'Bangladesh', 'Sri Lanka', 'Cambodia', 'Laos', 'Mongolia', 'Kazakhstan', 'Uzbekistan'
        ]

        for country in countries:
            if country.lower() in text.lower():
                return country

        return 'Unknown'

    def _get_location_name(self, latitude, longitude):
        """Get location name based on coordinates"""
        if not latitude or not longitude:
            return "Unknown Location"

        try:
            url = f"https://api.bigdatacloud.net/data/reverse-geocode-client?latitude={latitude}&longitude={longitude}&localityLanguage=en"
            response = requests.get(url, timeout=10)

            if response.status_code == 200:
                data = response.json()
                city = data.get('locality', data.get('city', ''))
                region = data.get('principalSubdivision', '')
                country = data.get('countryName', '')

                location_parts = [part for part in [city, region, country] if part]
                if location_parts:
                    return ', '.join(location_parts)
        except Exception as e:
            pass

        return f"Region ({latitude:.1f}°, {longitude:.1f}°)"

    def _extract_country_from_place(self, place):
        """Extract country from USGS place string"""
        if not place:
            return 'Unknown'

        parts = place.split(',')
        if len(parts) > 1:
            last_part = parts[-1].strip()

            country_mapping = {
                'Japan': 'Japan',
                'China': 'China',
                'Indonesia': 'Indonesia',
                'Philippines': 'Philippines',
                'Chile': 'Chile',
                'Peru': 'Peru',
                'Mexico': 'Mexico',
                'Turkey': 'Turkey',
                'Iran': 'Iran',
                'Greece': 'Greece',
                'Italy': 'Italy',
                'Alaska': 'United States',
                'California': 'United States',
                'Nevada': 'United States'
            }

            for key, value in country_mapping.items():
                if key.lower() in last_part.lower():
                    return value

            return last_part

        return self._extract_country_from_text('', place)

    def _determine_impact_level(self, distance, max_radius):
        """Determine impact level based on distance"""
        ratio = distance / max_radius

        if ratio <= 0.3:  # Within 30% range
            return "Critical"
        elif ratio <= 0.6:  # Within 60% range
            return "High"
        elif ratio <= 0.8:  # Within 80% range
            return "Medium"
        else:
            return "Low"

    def _calculate_distance(self, lat1, lon1, lat2, lon2):
        """Calculate distance between two points (kilometers)"""
        if not all([lat1, lon1, lat2, lon2]):
            return 999

        R = 6371  # Earth radius (kilometers)

        lat1_rad = math.radians(lat1)
        lat2_rad = math.radians(lat2)
        delta_lat = math.radians(lat2 - lat1)
        delta_lon = math.radians(lon2 - lon1)

        a = (math.sin(delta_lat/2) * math.sin(delta_lat/2) +
             math.cos(lat1_rad) * math.cos(lat2_rad) *
             math.sin(delta_lon/2) * math.sin(delta_lon/2))
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

        return R * c

    def _build_email_body(self, alerts, impact_radius, days, min_alert_level='Green'):
        """Build the email body content"""
        body = f"🏭 ENHANCED GDACS SUPPLIER IMPACT ANALYSIS REPORT\n"
        body += f"{'='*80}\n"
        body += f"📊 Total Disasters: {len(alerts)}\n"
        body += f"📅 Report Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
        body += f"⏰ Time Range: Past {days} days\n"
        body += f"🚨 Alert Level Filter: {self._get_alert_level_description(min_alert_level)}\n"
        body += f"🏭 Monitored Suppliers: {len(self.suppliers)}\n"
        body += f"📏 Impact Analysis Radius: {impact_radius}km\n"
        body += f"🔧 System: Enhanced GDACS with Complete Data Extraction\n\n"

        total_impacted_suppliers = 0

        # Analyze each disaster event
        for i, alert in enumerate(alerts, 1):
            coords = alert['coordinates']
            alert_emoji = {
                'Red': '🔴', 'Orange': '🟠', 'Yellow': '🟡', 'Green': '🟢'
            }.get(alert['alert_level'], '⚪')

            body += f"{i:2d}. {alert_emoji} {alert['event_type']} ({alert['event_type_short']}) - {alert['alert_level']} Alert\n"
            body += f"    📍 Location: {alert.get('location_name', alert['country'])}\n"
            body += f"    🏳️ Country: {alert['country']}\n"
            body += f"    📝 Title: {alert['title']}\n"

            if alert.get('severity') and alert['severity'] != 'Unknown':
                body += f"    📊 Severity: {alert['severity']}\n"

            if alert.get('magnitude'):
                body += f"    📈 Magnitude: {alert['magnitude']}\n"

            # Coordinates and impact analysis
            if coords['latitude'] and coords['longitude']:
                body += f"    🗺️ **COORDINATES: {coords['latitude']:.4f}°, {coords['longitude']:.4f}°**\n"
                body += f"    🌍 Google Maps: https://maps.google.com/?q={coords['latitude']},{coords['longitude']}\n"

                # Analyze supplier impact
                impacted_suppliers = self.analyze_supplier_impact(
                    coords['latitude'], coords['longitude'], impact_radius
                )

                if impacted_suppliers:
                    total_impacted_suppliers += len(impacted_suppliers)
                    body += f"    🚨 **IMPACTED SUPPLIERS: {len(impacted_suppliers)} companies**\n\n"

                    # Group by impact level
                    impact_groups = {}
                    for supplier in impacted_suppliers:
                        level = supplier['impact_level']
                        if level not in impact_groups:
                            impact_groups[level] = []
                        impact_groups[level].append(supplier)

                    # Display suppliers for each impact level
                    for impact_level in ['Critical', 'High', 'Medium', 'Low']:
                        if impact_level in impact_groups:
                            suppliers_in_level = impact_groups[impact_level]
                            impact_emoji = {
                                'Critical': '🔴', 'High': '🟠', 'Medium': '🟡', 'Low': '🟢'
                            }.get(impact_level, '⚪')

                            body += f"       {impact_emoji} {impact_level} Impact ({len(suppliers_in_level)} suppliers):\n"

                            for supplier in suppliers_in_level:
                                critical_marker = " ⚠️ CRITICAL SUPPLIER" if supplier['critical_level'] == 'High' else ""
                                body += f"         • {supplier['name']}{critical_marker}\n"
                                body += f"           📍 Address: {supplier['address']}\n"
                                body += f"           📏 Distance: {supplier['distance_km']:.1f}km from disaster\n"
                                body += f"           🏷️ Product Category: {supplier['product_category']}\n"
                                body += f"           🔧 Type: {supplier['type']}\n"
                                body += f"           📞 Contact Person: {supplier['contact_person']}\n"
                                body += f"           📧 Email: {supplier['email']}\n"
                                body += f"           ☎️ Phone: {supplier['phone']}\n\n"

                            body += "\n"
                else:
                    body += f"    ✅ **SUPPLIER IMPACT: No suppliers affected within {impact_radius}km**\n\n"
            else:
                body += f"    🗺️ **COORDINATES: Not available**\n"
                body += f"    🏭 **SUPPLIER ANALYSIS: Cannot perform without coordinates**\n\n"

            body += f"    ⏰ Published: {alert['pub_date']}\n"
            body += f"    🔗 Source: {alert['source']}\n"
            body += f"\n{'-'*70}\n\n"

        # Add summary
        body += self._build_summary_section(alerts, impact_radius, total_impacted_suppliers, days)

        return body

    def _build_summary_section(self, alerts, impact_radius, total_impacted_suppliers, days):
        """Build the summary section of the email"""
        body = "📊 ENHANCED IMPACT ANALYSIS SUMMARY:\n"
        body += f"{'='*60}\n"
        body += f"🚨 Total Impacted Suppliers: {total_impacted_suppliers}\n"
        body += f"🏭 Total Monitored Suppliers: {len(self.suppliers)}\n"
        body += f"⏰ Monitoring Period: {days} days\n"
        if len(self.suppliers) > 0:
            body += f"📈 Impact Rate: {(total_impacted_suppliers/len(self.suppliers)*100):.1f}% of supplier base\n"

        # Event type statistics
        event_type_stats = {}
        alert_level_stats = {}
        for alert in alerts:
            event_type = alert['event_type']
            alert_level = alert['alert_level']
            event_type_stats[event_type] = event_type_stats.get(event_type, 0) + 1
            alert_level_stats[alert_level] = alert_level_stats.get(alert_level, 0) + 1

        body += f"\n📈 EVENT TYPE BREAKDOWN:\n"
        for event_type, count in event_type_stats.items():
            body += f"   • {event_type}: {count} events\n"

        body += f"\n🚨 ALERT LEVEL BREAKDOWN:\n"
        for alert_level, count in alert_level_stats.items():
            emoji = {'Red': '🔴', 'Orange': '🟠', 'Yellow': '🟡', 'Green': '🟢'}.get(alert_level, '⚪')
            body += f"   {emoji} {alert_level}: {count} alerts\n"

        body += f"\n{'='*80}\n"
        body += "🤖 Enhanced Automated GDACS Supplier Impact Analysis System\n"
        body += "📡 Data sources: GDACS RSS (Enhanced), USGS, Google Maps Geocoding\n"
        body += "🔧 Complete GDACS attribute extraction with backup data sources\n"
        body += "⚠️ Please verify all information and contact suppliers directly.\n"
        body += "🔄 Recommended: Activate backup suppliers for affected categories.\n"

        return body

    def _count_total_impacted_suppliers(self, alerts, impact_radius):
        """Count total impacted suppliers across all alerts"""
        total = 0
        for alert in alerts:
            coords = alert['coordinates']
            if coords['latitude'] and coords['longitude']:
                impacted = self.analyze_supplier_impact(coords['latitude'], coords['longitude'], impact_radius)
                total += len(impacted)
        return total

    def _send_no_alerts_notification(self, days, min_alert_level='Green'):
        """Send no alerts notification and save JSON"""
        try:
            # Generate JSON for no alerts case
            json_data = self._generate_no_alerts_json(days, min_alert_level)
            json_saved = self._save_json_report(json_data)

            body = f"🟢 ENHANCED GDACS SUPPLIER STATUS: ALL CLEAR\n"
            body += f"{'='*60}\n"
            body += f"📅 Report Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
            body += f"⏰ Monitoring Period: Past {days} days\n"
            body += f"🚨 Alert Level Filter: {self._get_alert_level_description(min_alert_level)}\n"
            body += f"🏭 Monitored Suppliers: {len(self.suppliers)}\n"
            body += f"✅ No significant disasters detected at {min_alert_level}+ level\n"
            body += f"🌍 Global disaster monitoring is active\n"
            body += f"🏭 All suppliers are currently safe from disasters\n"

            msg = MIMEText(body, 'plain', 'utf-8')
            msg['Subject'] = Header(f"🟢 Enhanced Supplier Status: All Clear ({days} days, {min_alert_level}+ level)", 'utf-8')
            msg['From'] = formataddr(("Enhanced GDACS Supplier Impact System", self.email_config['email']))
            msg['To'] = formataddr(("Supply Chain Manager", self.email_config['recipient']))

            server = smtplib.SMTP(self.email_config['server'], self.email_config['port'])
            server.starttls()
            server.login(self.email_config['email'], self.email_config['password'])
            server.sendmail(self.email_config['email'], self.email_config['recipient'], msg.as_string())
            server.quit()

            print("📧 Enhanced 'All Clear' notification sent!")
            if json_saved:
                print("💾 JSON report saved successfully!")
            return True

        except Exception as e:
            print(f"❌ Failed to send notification: {e}")
            return False

    def _generate_json_report(self, alerts, impact_radius, days, min_alert_level='Green'):
        """Generate comprehensive JSON report"""
        report_data = {
            "report_metadata": {
                "generation_time": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                "system_name": "Enhanced GDACS Supplier Impact Analysis System",
                "version": "2.0",
                "monitoring_period_days": days,
                "minimum_alert_level": min_alert_level,
                "alert_level_description": self._get_alert_level_description(min_alert_level),
                "impact_analysis_radius_km": impact_radius,
                "total_disasters": len(alerts),
                "total_monitored_suppliers": len(self.suppliers)
            },
            "system_statistics": {
                "geocoded_suppliers": len([s for s in self.suppliers if s['geocoded']]),
                "total_impacted_suppliers": self._count_total_impacted_suppliers(alerts, impact_radius),
                "impact_rate_percentage": (self._count_total_impacted_suppliers(alerts, impact_radius) / len(self.suppliers) * 100) if len(self.suppliers) > 0 else 0
            },
            "disaster_alerts": [],
            "supplier_impact_analysis": {},
            "event_statistics": {
                "by_event_type": {},
                "by_alert_level": {},
                "by_country": {}
            },
            "all_suppliers": []
        }

        # Add all suppliers information
        for supplier in self.suppliers:
            supplier_data = {
                "id": supplier['id'],
                "name": supplier['name'],
                "address": supplier['address'],
                "country": supplier['country'],
                "contact_person": supplier['contact_person'],
                "phone": supplier['phone'],
                "email": supplier['email'],
                "product_category": supplier['product_category'],
                "critical_level": supplier['critical_level'],
                "type": supplier['type'],
                "coordinates": {
                    "latitude": supplier['latitude'],
                    "longitude": supplier['longitude'],
                    "geocoded": supplier['geocoded']
                }
            }
            report_data["all_suppliers"].append(supplier_data)

        # Process each disaster alert
        for i, alert in enumerate(alerts):
            alert_data = {
                "alert_id": i + 1,
                "basic_info": {
                    "title": alert['title'],
                    "description": alert['description'],
                    "event_type": alert['event_type'],
                    "event_type_short": alert['event_type_short'],
                    "alert_level": alert['alert_level'],
                    "country": alert['country'],
                    "location_name": alert.get('location_name', 'Unknown Location'),
                    "publication_date": alert['pub_date'],
                    "source": alert['source']
                },
                "coordinates": alert['coordinates'],
                "additional_attributes": {
                    "event_id": alert.get('event_id'),
                    "event_name": alert.get('event_name'),
                    "severity": alert.get('severity'),
                    "population": alert.get('population'),
                    "vulnerability": alert.get('vulnerability'),
                    "from_date": alert.get('from_date').strftime('%Y-%m-%d %H:%M:%S') if alert.get('from_date') else None,
                    "to_date": alert.get('to_date').strftime('%Y-%m-%d %H:%M:%S') if alert.get('to_date') else None,
                    "duration_in_week": alert.get('duration_in_week'),
                    "is_current": alert.get('is_current'),
                    "temporary": alert.get('temporary'),
                    "magnitude": alert.get('magnitude'),
                    "depth": alert.get('depth')
                },
                "links": {
                    "source_link": alert.get('link', ''),
                    "icon_url": alert.get('icon_url', ''),
                    "google_maps": f"https://maps.google.com/?q={alert['coordinates']['latitude']},{alert['coordinates']['longitude']}" if alert['coordinates']['latitude'] and alert['coordinates']['longitude'] else ""
                },
                "impacted_suppliers": []
            }

            # Analyze supplier impact for this disaster
            if alert['coordinates']['latitude'] and alert['coordinates']['longitude']:
                impacted_suppliers = self.analyze_supplier_impact(
                    alert['coordinates']['latitude'],
                    alert['coordinates']['longitude'],
                    impact_radius
                )

                # Group suppliers by impact level
                impact_groups = {}
                for supplier in impacted_suppliers:
                    level = supplier['impact_level']
                    if level not in impact_groups:
                        impact_groups[level] = []

                    supplier_impact_data = {
                        "supplier_id": supplier['id'],
                        "name": supplier['name'],
                        "address": supplier['address'],
                        "country": supplier['country'],
                        "contact_info": {
                            "contact_person": supplier['contact_person'],
                            "phone": supplier['phone'],
                            "email": supplier['email']
                        },
                        "business_info": {
                            "product_category": supplier['product_category'],
                            "critical_level": supplier['critical_level'],
                            "type": supplier['type']
                        },
                        "coordinates": {
                            "latitude": supplier['latitude'],
                            "longitude": supplier['longitude']
                        },
                        "impact_analysis": {
                            "distance_km": round(supplier['distance_km'], 2),
                            "impact_level": supplier['impact_level']
                        }
                    }

                    impact_groups[level].append(supplier_impact_data)
                    alert_data["impacted_suppliers"].append(supplier_impact_data)

                alert_data["impact_summary"] = {
                    "total_impacted": len(impacted_suppliers),
                    "by_impact_level": {level: len(suppliers) for level, suppliers in impact_groups.items()}
                }
            else:
                alert_data["impact_summary"] = {
                    "total_impacted": 0,
                    "reason": "No coordinates available for impact analysis"
                }

            report_data["disaster_alerts"].append(alert_data)

            # Update statistics
            event_type = alert['event_type']
            alert_level = alert['alert_level']
            country = alert['country']

            report_data["event_statistics"]["by_event_type"][event_type] = \
                report_data["event_statistics"]["by_event_type"].get(event_type, 0) + 1
            report_data["event_statistics"]["by_alert_level"][alert_level] = \
                report_data["event_statistics"]["by_alert_level"].get(alert_level, 0) + 1
            report_data["event_statistics"]["by_country"][country] = \
                report_data["event_statistics"]["by_country"].get(country, 0) + 1

        return report_data

    def _generate_no_alerts_json(self, days, min_alert_level='Green'):
        """Generate JSON report for no alerts case"""
        report_data = {
            "report_metadata": {
                "generation_time": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                "system_name": "Enhanced GDACS Supplier Impact Analysis System",
                "version": "2.0",
                "monitoring_period_days": days,
                "minimum_alert_level": min_alert_level,
                "alert_level_description": self._get_alert_level_description(min_alert_level),
                "total_disasters": 0,
                "total_monitored_suppliers": len(self.suppliers),
                "status": "ALL_CLEAR"
            },
            "system_statistics": {
                "geocoded_suppliers": len([s for s in self.suppliers if s['geocoded']]),
                "total_impacted_suppliers": 0,
                "impact_rate_percentage": 0
            },
            "disaster_alerts": [],
            "supplier_impact_analysis": {
                "summary": f"No disasters detected at {min_alert_level}+ level in the monitoring period"
            },
            "event_statistics": {
                "by_event_type": {},
                "by_alert_level": {},
                "by_country": {}
            },
            "all_suppliers": []
        }

        # Add all suppliers information
        for supplier in self.suppliers:
            supplier_data = {
                "id": supplier['id'],
                "name": supplier['name'],
                "address": supplier['address'],
                "country": supplier['country'],
                "contact_person": supplier['contact_person'],
                "phone": supplier['phone'],
                "email": supplier['email'],
                "product_category": supplier['product_category'],
                "critical_level": supplier['critical_level'],
                "type": supplier['type'],
                "coordinates": {
                    "latitude": supplier['latitude'],
                    "longitude": supplier['longitude'],
                    "geocoded": supplier['geocoded']
                },
                "status": "SAFE"
            }
            report_data["all_suppliers"].append(supplier_data)

        return report_data

    def _save_json_report(self, json_data):
        """Save JSON report to file"""
        try:
            # Generate filename with timestamp
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            filename = f"gdacs_supplier_impact_report_{timestamp}.json"

            # Save to current directory (Google Colab)
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(json_data, f, indent=2, ensure_ascii=False, default=str)

            print(f"💾 JSON report saved as: {filename}")
            print(f"📊 Report contains:")
            print(f"   • {json_data['report_metadata']['total_disasters']} disaster alerts")
            print(f"   • {json_data['report_metadata']['total_monitored_suppliers']} suppliers")
            print(f"   • {json_data['system_statistics']['total_impacted_suppliers']} impacted suppliers")
            print(f"   • Monitoring period: {json_data['report_metadata']['monitoring_period_days']} days")

            return True

        except Exception as e:
            print(f"❌ Failed to save JSON report: {e}")
            return False


def main():
    """Main program"""
    print("🏭 ENHANCED GDACS SUPPLIER IMPACT ANALYSIS SYSTEM")
    print("="*90)
    print("📧 Monitors disasters with complete GDACS data and analyzes supplier impact")
    print("🔍 Enhanced Features:")
    print("   • Complete GDACS attribute extraction (all 25+ fields)")
    print("   • Multiple coordinate parsing methods")
    print("   • Enhanced event classification")
    print("   • Backup data sources (USGS)")
    print("   • Improved geocoding and mapping")
    print("   • Supplier type analysis support")
    print("   • User-defined monitoring period")
    print("="*90)

    # Initialize enhanced system
    system = EnhancedGDACSSupplierImpactSystem()

    # Check supplier CSV file
    csv_file = "merged_full_dataset.csv"
    if not os.path.exists(csv_file):
        print(f"\n📝 Supplier CSV file not found: {csv_file}")
        print("❌ Cannot proceed without supplier data")
        print("Please ensure your CSV file is named 'merged_full_dataset.csv' and contains the required columns:")
        print("   id, name, address, country, contact_person, phone, email, product_category, critical_level, type")
        return

    # Load supplier data
    print(f"\n🏭 Loading suppliers from {csv_file}...")
    print("🔍 Validating CSV format...")
    system.validate_csv_format(csv_file)

    if not system.load_suppliers_from_csv(csv_file):
        print("❌ Failed to load supplier data")
        print("\n🔧 TROUBLESHOOTING:")
        print("   1. Check if addresses are properly quoted or use alternative delimiter")
        print("   2. Ensure all required fields (id, name, address, country) are present")
        print("   3. Use UTF-8 encoding when saving the CSV file")
        return

    # Gmail configuration
    print("\n📧 Gmail Configuration:")
    gmail = input("📧 Enter your Gmail address: ").strip()
    password = input("🔑 Enter your Gmail App Password: ").strip()
    recipient = input("📥 Enter recipient email address: ").strip()

    if not gmail or not password or not recipient:
        print("❌ All fields are required!")
        return

    system.setup_gmail(gmail, password, recipient)

    # Google Maps API configuration
    print("\n🗺️ Google Maps API Configuration:")
    print("📝 Required for address geocoding (converting addresses to coordinates)")
    print("   1. Go to https://console.cloud.google.com/")
    print("   2. Enable 'Geocoding API'")
    print("   3. Create an API key")

    api_key = input("🔑 Enter your Google Maps API key: ").strip()

    if api_key:
        system.setup_google_maps(api_key)

        # Address geocoding
        print("\n🗺️ Converting supplier addresses to coordinates...")
        geocoded = system.geocode_suppliers()

        if not geocoded:
            print("⚠️ Geocoding failed - impact analysis will be limited")
    else:
        print("❌ Google Maps API key required for supplier impact analysis")
        return

    # Test email
    print("\n📬 Testing email connection...")
    if not system.send_test_email():
        print("❌ Email test failed. Please check your settings.")
        return

    # User-defined monitoring period
    print("\n⏰ Monitoring Period Configuration:")
    print("📝 Set how many days back to search for disasters")
    days_input = input("📅 Enter number of days to monitor (default: 7): ").strip()

    try:
        monitoring_days = int(days_input) if days_input else 7
        if monitoring_days < 1:
            print("⚠️ Days must be at least 1, using default value: 7")
            monitoring_days = 7
        elif monitoring_days > 30:
            print("⚠️ Maximum 30 days supported, setting to 30")
            monitoring_days = 30
    except ValueError:
        print("⚠️ Invalid input, using default value: 7 days")
        monitoring_days = 7

    print(f"📊 Monitoring period set to: {monitoring_days} days")

    # Alert level configuration
    print("\n🚨 Alert Level Configuration:")
    print("📝 Choose minimum alert level to monitor (GDACS Standard)")
    print("   1. 🟢 Green and above (All alerts - includes minor events)")
    print("   2. 🟠 Orange and above (High to severe events)")
    print("   3. 🔴 Red only (Severe events only)")

    level_choice = input("🔢 Enter your choice (1-3, default: 1): ").strip()

    level_mapping = {
        '1': 'Green',
        '2': 'Orange',
        '3': 'Red'
    }

    min_alert_level = level_mapping.get(level_choice, 'Green')
    print(f"🚨 Alert level filter set to: {system._get_alert_level_description(min_alert_level)}")

    # Get enhanced disaster alerts
    print(f"\n🔍 Fetching enhanced disaster alerts from past {monitoring_days} days...")
    alerts = system.get_enhanced_gdacs_alerts(days=monitoring_days, min_alert_level=min_alert_level)

    # Set impact radius
    impact_radius = input(f"\n📏 Enter impact analysis radius in km (default: 100): ").strip()
    try:
        impact_radius = int(impact_radius) if impact_radius else 100
    except:
        impact_radius = 100

    # Send enhanced supplier impact analysis report
    print(f"\n📊 Analyzing enhanced supplier impact (radius: {impact_radius}km, period: {monitoring_days} days, level: {min_alert_level}+)...")
    success = system.send_enhanced_supplier_impact_alert(alerts, impact_radius, monitoring_days, min_alert_level)

    if success:
        print("✅ Enhanced supplier impact analysis completed successfully!")
    else:
        print("❌ Enhanced supplier impact analysis encountered errors.")

    print(f"\n📈 Enhanced Summary:")
    print(f"   📧 Email system: ✅ Working")
    print(f"   🗺️ Google Maps API: ✅ Working")
    print(f"   🏭 Suppliers loaded: {len(system.suppliers)}")
    print(f"   ⏰ Monitoring period: {monitoring_days} days")
    print(f"   🚨 Alert level filter: {min_alert_level}+ ({system._get_alert_level_description(min_alert_level)})")
    print(f"   🌍 Enhanced disasters found: {len(alerts)}")
    print(f"   🔧 GDACS attributes extracted: ✅ Complete")
    print(f"   📬 Enhanced report sent: {'✅ Yes' if success else '❌ No'}")
    print(f"   💾 JSON report saved: ✅ Yes")
    print(f"\n📂 Files generated:")
    print(f"   • JSON report: gdacs_supplier_impact_report_[timestamp].json")
    print(f"   • Location: Current Google Colab directory")
    print(f"\n🔍 JSON report contains:")
    print(f"   • Complete disaster alert details")
    print(f"   • Full supplier impact analysis")
    print(f"   • Statistical summaries")
    print(f"   • All supplier information")
    print(f"   • Coordinates and mapping data")
    print(f"   • Alert level filtering information")


if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n\n👋 Program interrupted by user. Goodbye!")
    except Exception as e:
        print(f"\n❌ Unexpected error: {e}")
        print("Please restart the program.")

🏭 ENHANCED GDACS SUPPLIER IMPACT ANALYSIS SYSTEM
📧 Monitors disasters with complete GDACS data and analyzes supplier impact
🔍 Enhanced Features:
   • Complete GDACS attribute extraction (all 25+ fields)
   • Multiple coordinate parsing methods
   • Enhanced event classification
   • Backup data sources (USGS)
   • Improved geocoding and mapping
   • Supplier type analysis support
   • User-defined monitoring period
✅ Enhanced GDACS Supplier Impact Analysis System Initialized
🏭 Features: Complete GDACS data extraction + Supplier impact analysis
📊 Enhanced with full GDACS attributes support

📝 Supplier CSV file not found: merged_full_dataset.csv
❌ Cannot proceed without supplier data
Please ensure your CSV file is named 'merged_full_dataset.csv' and contains the required columns:
   id, name, address, country, contact_person, phone, email, product_category, critical_level, type
